In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [3]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, labels, img_dir, transform=None, target_transform=None):
        self.img_labels = labels
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx,0])
        image = (Image.open(img_path))
        label = self.img_labels.iloc[idx,1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return [image, label]

In [4]:
df=pd.read_csv('../input/100-bird-species/birds/birds.csv')
'''labels_train=df.labels[0:39364]
labels_test = df.labels[39364:40739]
labels_valid = df.labels[40739:42114]'''
new=df.drop(['Unnamed: 0','data set'],axis=1)
new['filepaths']=new['filepaths'].str.replace('\\','/')
label_encoder = preprocessing.LabelEncoder()
new['labels']= label_encoder.fit_transform(new['labels'])
new


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


,filepaths,labels
0,train/AFRICAN CROWNED CRANE/001.jpg,0
1,train/AFRICAN CROWNED CRANE/002.jpg,0
2,train/AFRICAN CROWNED CRANE/003.jpg,0
3,train/AFRICAN CROWNED CRANE/004.jpg,0
4,train/AFRICAN CROWNED CRANE/005.jpg,0
...,...,...
42109,valid/YELLOW HEADED BLACKBIRD/1.jpg,274
42110,valid/YELLOW HEADED BLACKBIRD/2.jpg,274
42111,valid/YELLOW HEADED BLACKBIRD/3.jpg,274
42112,valid/YELLOW HEADED BLACKBIRD/4.jpg,274


In [5]:

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = CustomImageDataset(new[:][0:39364],'../input/100-bird-species/birds/',transform=transform)
test_data = CustomImageDataset(new[:][39364:40739],'../input/100-bird-species/birds/',transform=transform)
valid_data = CustomImageDataset(new[:][40739:442114],'../input/100-bird-species/birds/',transform=transform)

training_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)
testing_dataloader = DataLoader(test_data,batch_size=1375,shuffle=True)
valid_dataloader = DataLoader(valid_data,batch_size=64,shuffle=False)

In [26]:
#Architecture
class CusNet(nn.Module):
  def __init__(self):
    super(CusNet,self).__init__()
    self.flatten = nn.Flatten()
    self.convolve = nn.Sequential(
        nn.Conv2d(3,96,kernel_size=(11,11),stride=4),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2),
        nn.Conv2d(96,256,kernel_size=(5,5),stride=1,padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2),
        nn.Conv2d(256,384,kernel_size=(3,3),stride=1,padding=1),
        nn.BatchNorm2d(384),
        nn.ReLU(),
        nn.Conv2d(384,384,kernel_size=(3,3),stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(384,256,kernel_size=(3,3),stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(3,3),stride=2)
    )
    self.classification = nn.Sequential(
        #nn.Dropout(p=0.5),
        nn.Linear(9216,4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        #nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        nn.Linear(4096,275)
    )
  def forward(self,x):
    x=self.convolve(x)
    x=self.flatten(x)
    return self.classification(x)

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.00)


model = CusNet()
if torch.cuda.is_available():
  model.cuda()

model.apply(init_weights)
optimizer_1 = optim.Adam(model.parameters(),lr=0.003,weight_decay=1e-5)
optimizer_2 = optim.SGD(model.parameters(),lr=0.05,weight_decay=0.0,momentum=0.9)
optimizer_3= optim.Adam(model.parameters(),lr=0.005,weight_decay=1e-5)
loss_fn_1 = nn.CrossEntropyLoss()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [16]:
epoch = 10
for i in range(epoch):
  batch=0
  loss1=0
  for x,y in training_dataloader:
    batch+=1
    if torch.cuda.is_available():
      x,y = x.cuda(),y.cuda()
    
    optimizer_1.zero_grad()
    output = model(x)
    loss = loss_fn_1(output,y)
    loss.backward()
    optimizer_1.step()
    loss1 += loss.item()
  loss1/=batch
  print(f"avg loss in {i+1} epoch is {loss1}")


  batch1=0
  correct =0
  with torch.no_grad():
    for x,y in valid_dataloader:
      if torch.cuda.is_available():
        x,y = x.cuda(),y.cuda()
      batch1+=1
      pred = model(x)
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct/=len(valid_dataloader.dataset)
    print(f" accuracy is {correct*100}")
    

avg loss in 1 epoch is 5.845470808543168
 accuracy is 7.636363636363637
avg loss in 2 epoch is 3.5501636475711673
 accuracy is 31.418181818181818
avg loss in 3 epoch is 2.3496103371892656
 accuracy is 45.018181818181816
avg loss in 4 epoch is 1.6998608098788694
 accuracy is 51.92727272727272
avg loss in 5 epoch is 1.2648443902854796
 accuracy is 54.90909090909091
avg loss in 6 epoch is 0.9641565233469009
 accuracy is 59.27272727272728
avg loss in 7 epoch is 0.7333882040791697
 accuracy is 60.58181818181818
avg loss in 8 epoch is 0.5623671172259302
 accuracy is 62.32727272727273
avg loss in 9 epoch is 0.4667882358824665
 accuracy is 63.78181818181818
avg loss in 10 epoch is 0.35036234766873836
 accuracy is 63.63636363636363


In [19]:
correct =0
with torch.no_grad():
    for x,y in testing_dataloader:
      if torch.cuda.is_available():
        x,y = x.cuda(),y.cuda()
      pred = model(x)
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct/=len(testing_dataloader.dataset)
    print(f" accuracy is {correct*100}")

 accuracy is 79.49090909090908


In [20]:
torch.save(model,'model_3.pth')